In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
import seaborn as sb
import pandas as pd

from MDAnalysis.analysis import pca as mda_pca
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix
from sklearn.decomposition import PCA


%matplotlib inline

import nglview as nv

import warnings
warnings.filterwarnings('ignore')